In [1]:
import os
import smtplib
import dotenv
from dotenv import load_dotenv
load_dotenv()
import pandas as pd

In [2]:
df = pd.read_csv('products_ready.csv')

In [3]:
df

,Item,Price
0,"Apple MacBook Pro 16"" - Silber 2019 MVVL2D/A i...",5124.43
1,"Apple MacBook Pro 16"" - Silber 2019 MVVM2D/A i...",3057.92
2,"Apple MacBook Pro 16"" - Space Grau 2019 MVVK2D...",3057.92
3,"Nuevo Apple MacBook Pro (de 16 pulgadas, 16 GB...",2859.00
4,"Apple - MacBook Pro 15"" (All-in-One Desktop PC...",2699.00


Dejo las imagenes para el PDF

In [4]:
df_email = df[['Price','Item']]

In [5]:
df_email

,Price,Item
0,5124.43,"Apple MacBook Pro 16"" - Silber 2019 MVVL2D/A i..."
1,3057.92,"Apple MacBook Pro 16"" - Silber 2019 MVVM2D/A i..."
2,3057.92,"Apple MacBook Pro 16"" - Space Grau 2019 MVVK2D..."
3,2859.00,"Nuevo Apple MacBook Pro (de 16 pulgadas, 16 GB..."
4,2699.00,"Apple - MacBook Pro 15"" (All-in-One Desktop PC..."


In [6]:
#df_email = df_email[['o','Price','oo','Item']]

In [7]:
df_email.to_csv('Email_1.csv',index=False)

In [8]:
sender_email = os.getenv('emailP')
receiver_email = os.getenv('email')
password = os.getenv('PasswordP')
password

'Confinamiento2020'

In [38]:
mi_imagen='/Users/albertjoselobera/Ironhack/Projects/Personal_Projects/Personal_Projects/IMG'

In [60]:
# Import smtplib for the actual sending function
import smtplib

# Here are the email package modules we'll need
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

def email():    
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = "Pythonium: Lo mejor que he encontrado"
    html =f"""\
    <html>
    <head></head>
    <body>
        <h3>¡Hola, !</h3>
        <p>Muchas garcias por encargarme mi primera misión, he estado investigando por Amazon sobre: <br>
        De todos lo que he visto, he seleccionado los 5 mejores y te los adjunto en este email.<br>
        Acuérdate de indicarme el número del artículo que más te guste, y del precio que estarías interesado.<p>
       

        
        Seguimos en contacto,<br>
        Pythonium<br><br>
        </p>

    </body>
    </html>
    """
    # Assume we know that the image files are all in PNG format
    
    # Open the files in binary mode.  Let the MIMEImage class automatically
    # guess the specific image type.
    
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    fileToSend = 'src/CSV/Products.csv'
    ctype, encoding = mimetypes.guess_type(fileToSend)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "html":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    fp = open('IMG/pythonium.png', 'rb')
    img = MIMEImage(fp.read())
    fp.close()
    msg.attach(img)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(sender_email,password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    print('Email sent')
    server.quit()


In [61]:
email()

Email sent


In [39]:
import os
import smtplib
import dotenv
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.utils import make_msgid

def email():
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = "Pythonium: Lo mejor que he encontrado"
    html =f"""\
    <html>
    <head></head>
    <body>
        <h3>¡Hola, !</h3>
        <p>Muchas garcias por encargarme mi primera misión, he estado investigando por Amazon sobre: <br>
        De todos lo que he visto, he seleccionado los 5 mejores y te los adjunto en este email.<br>
        Acuérdate de indicarme el número del artículo que más te guste, y del precio que estarías interesado.<p>
       

        
        Seguimos en contacto,<br>
        Pythonium<br><br>
        </p>
    <p><img src="cid:0"></p>'
    <figure class="wp-block-image size-large is-resized"><img src="https://github.com/AlbertJlobera/Personal_Projects/blob/master/IMG/pythonium.jpg" width="524" height="347"/></figure> 
    </body>
    </html>
    """
    
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    fileToSend = 'src/CSV/Products.csv'
    myImage = 'IMG/pythonium.jpg'
    ctype, encoding = mimetypes.guess_type(fileToSend,myImage)
    maintype, subtype = ctype.split("/", 1)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    
    if maintype == "html":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "image":
        fp = open(myImage, "rb")
        attachment = MIMEImage(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(sender_email,password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
 
    server.quit()


In [40]:
email()

IsADirectoryError: [Errno 21] Is a directory: '/Users/albertjoselobera/Ironhack/Projects/Personal_Projects/Personal_Projects/IMG'

In [1]:
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# me == my email address
# you == recipient's email address
sender_email = os.getenv('emailP')
receiver_email = os.getenv('email')

# Create message container - the correct MIME type is multipart/alternative.
msg = MIMEMultipart('alternative')
msg['Subject'] = 'Pythonium'
msg['From'] = sender_email
msg['To'] = receiver_email

# Create the body of the message (a plain-text and an HTML version).

html = """\
<html>
  <head></head>
  <body>
    <p>Hi!<br>
       How are you?<br>
       Here is the <a href="https://www.python.org">link</a> you wanted.
    </p>
  </body>
</html>
"""

# Record the MIME types of both parts - text/plain and text/html.

part2 = MIMEText(html, 'html')

# Attach parts into message container.
# According to RFC 2046, the last part of a multipart message, in this case
# the HTML message, is best and preferred.

msg.attach(part2)

# Send the message via local SMTP server.
s = smtplib.SMTP('localhost')
# sendmail function takes 3 arguments: sender's address, recipient's address
# and message to send - here it is sent as one string.
s.sendmail(sender_email, receiver_email, msg.as_string())
s.quit()

NameError: name 'os' is not defined

In [9]:
def send_mail():

    
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')


    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    
    server.login(sender_email,password)
    
    subject = 'Soy Pythonium, encantado de conocerte'
    
    body = f'''Buenos día,\nAquí tienes el producto que me pediste, 
    te mando una lista de los cinco productos más caros que he econtrado.
    \nDime cual quieres indicando el número que te aparece delante de cada artículo, van del 0 al 4, 
    necesitaría tambien el precio al cual estarías interesado.\n\n\n{df_email}\n\n\n Te avisaré cuando el precio baje, 
    si no me indicas un precio estimado te avisaré cuando baje, aun que sea solo un poco :).\n\n
        Recuerda que me encontrarás en tu terminal con el comando ......\n\nTen un buen día!,\nPythonium'''
    
    msg = f'Subject:{subject}\n\n{body}'.encode()
    
    
    server.sendmail(
        sender_email,
        receiver_email,
        msg
    )
    
    print('Hey email has been sent!')
    
    server.quit()



Hey email has been sent!


# check for PDF

In [11]:
  
import smtplib
import email,ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText




def send_email1():
    subject="Check out my Proyect 2: Data Pipeline!"
    body = """\
    Hola,
    Adjunto vas a encontrar una gráfica con los 10 tenistas con mejores resultados en el torneo y año que seleccionado.
    Además, estos son los ganadores de Grand Slams de los últimos años (2017,2018,2019) con el número total de titulos(Round), media de juegos por partido y media de sets jugados: {text}
    Por último,estos son los datos personales de cada ganador: Nacionalidad, Fecha de Nacimiento, Altura, Ganancias: 
    """
    sender_email = os.getenv('email')
    receiver_email = os.getenv('email')
    password = os.getenv('Password')

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))
    filename = "Email_1.csv"
    with open(filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )
    message.attach(part)
    

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email,msg)
        